# Brand new No BFS approach


In [18]:
import carla
import math
import numpy as np

def precompute_driving_waypoints(carla_map, step=2.0):
    """
    Precompute all driving waypoints for the map at given resolution.
    Call this once and reuse the result.
    """
    all_wps = carla_map.generate_waypoints(distance=step)
    driving_wps = [wp for wp in all_wps if wp.lane_type == carla.LaneType.Driving]
    return driving_wps

def get_local_points_from_precomputed(
    driving_wps,
    ego_loc,
    radius=60.0,
    n_points=200,
    step=2.0,
):
    """
    No BFS. Just:
      - filter precomputed waypoints by radius
      - add center points
      - add lane marking points
      - sort by distance and keep up to n_points
    """
    candidates = []

    # 1) Center points
    for wp in driving_wps:
        loc = wp.transform.location
        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)
        if dist > radius:
            continue

        yaw_rad = math.radians(wp.transform.rotation.yaw)
        half_length = step * 0.5
        half_width = wp.lane_width * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        geom_type_code = 0.0
        lm_type = 0.0

        candidates.append(
            (
                dist,
                loc.x, loc.y,
                yaw_rad,
                half_length, half_width,
                lane_type_code, road_id,
                geom_type_code, lm_type,
            )
        )

    # 2) Lane drawings (left/right markings) for the same wps
    for wp in driving_wps:
        loc = wp.transform.location
        dx0 = loc.x - ego_loc.x
        dy0 = loc.y - ego_loc.y
        base_dist = math.hypot(dx0, dy0)
        if base_dist > radius:
            continue

        yaw_rad = math.radians(wp.transform.rotation.yaw)
        nx = -math.sin(yaw_rad)
        ny =  math.cos(yaw_rad)
        half_width = wp.lane_width * 0.5
        half_length = step * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        # left marking
        lm_left = wp.left_lane_marking
        if lm_left is not None and lm_left.type != carla.LaneMarkingType.NONE:
            bx = loc.x + nx * half_width
            by = loc.y + ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 1.0
                lm_type = float(int(lm_left.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

        # right marking
        lm_right = wp.right_lane_marking
        if lm_right is not None and lm_right.type != carla.LaneMarkingType.NONE:
            bx = loc.x - nx * half_width
            by = loc.y - ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 2.0
                lm_type = float(int(lm_right.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

    D = 9
    points = np.zeros((n_points, D), dtype=np.float32)
    if not candidates:
        return points

    candidates.sort(key=lambda t: t[0])
    selected = candidates[:n_points]

    for i, (_, x, y, yaw, hlen, hwid,
            lane_t, rid, geom_t, lm_type) in enumerate(selected):
        points[i, :] = [x, y, yaw, hlen, hwid, lane_t, rid, geom_t, lm_type]

    return points


In [19]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
carla_map = world.get_map()

# Precompute once
driving_wps = precompute_driving_waypoints(carla_map, step=2.0)

# Pick a fixed ego point
all_wps = carla_map.generate_waypoints(10.0)
ego_wp = all_wps[0]
ego_loc = carla.Location(x=-44.0,y=129.7)

for r in [21.0, 22.0, 23.0]:
    pts = get_local_points_from_precomputed(
        driving_wps,
        ego_loc,
        radius=r,
        n_points=5000,
        step=2.0,
    )
    valid = (pts[:,0] != 0) | (pts[:,1] != 0)
    print(f"radius {r}: {valid.sum()} points")


radius 21.0: 151 points
radius 22.0: 161 points
radius 23.0: 175 points


# Complete radius sampling code

In [20]:
import carla
import math
import numpy as np
import random
import time


In [21]:
def precompute_driving_waypoints(carla_map, step=2.0):
    """
    Precompute all driving waypoints for the map at given resolution.
    Call this once and reuse the result.
    """
    all_wps = carla_map.generate_waypoints(distance=step)
    driving_wps = [wp for wp in all_wps if wp.lane_type == carla.LaneType.Driving]
    print(f"Precomputed {len(driving_wps)} driving waypoints (step={step})")
    return driving_wps

In [22]:
def precompute_landmarks(carla_map):
    """
    Precompute all landmarks from the map.
    """
    all_landmarks = carla_map.get_all_landmarks()
    print(f"Precomputed {len(all_landmarks)} landmarks")
    return all_landmarks


In [23]:
def precompute_crosswalks(carla_map):
    """
    Precompute all crosswalks from the map.
    """
    all_crosswalks = carla_map.get_crosswalks()
    print(f"Precomputed {len(all_crosswalks)} crosswalks")
    return all_crosswalks



In [57]:
def classify_landmark(lm: carla.Landmark) -> str:
    """
    Return a small semantic tag for a CARLA landmark.
    """
    t = (lm.type or "").strip()
    name = (lm.name or "").lower()

    if t == "206" or "stop" in name:
        return "stop_sign"
    if t == "205" or "yield" in name:
        return "yield_sign"
    if t == "1000001" or "signal" in name:
        return "traffic_light"

    return "other"

In [58]:
def get_local_points_from_precomputed_knearest(
    driving_wps,
    ego_loc,
    radius=60.0,
    n_points=200,
    step=2.0,
    all_landmarks=None,
    all_crosswalks=None,
):
    """
    Using precomputed driving waypoints:
      - Find waypoints whose *center* is within radius of ego_loc.
      - Build lane center points for them.
      - Build lane marking points (left/right) from them.
      - Keep only those boundary points whose own position is within radius.
      - Sort all candidates by distance and keep up to n_points.

    Returns:
        points: np.ndarray of shape (n_points, 9), rows:

            [x_world, y_world, yaw_rad,
             half_length, half_width,
             lane_type_code, road_id,
             geom_type_code, lane_mark_type_code]

        geom_type_code:
            0 = lane center
            1 = left lane marking
            2 = right lane marking

        lane_mark_type_code:
            0 for centers
            numeric code (float) for lane marking types for boundary points
    """
    candidates = []

    # First: select only waypoints whose center is within radius
    wps_near = []
    for wp in driving_wps:
        loc = wp.transform.location
        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)
        if dist <= radius:
            wps_near.append((wp, loc, dist))

    # 1) Lane center points
    for wp, loc, dist in wps_near:
        yaw_rad = math.radians(wp.transform.rotation.yaw)
        half_length = step * 0.5
        half_width = wp.lane_width * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        geom_type_code = 0.0
        lm_type = 0.0

        candidates.append(
            (
                dist,
                loc.x, loc.y,
                yaw_rad,
                half_length, half_width,
                lane_type_code, road_id,
                geom_type_code, lm_type,
            )
        )

    # 2) Lane marking points (left/right), built from the same nearby waypoints
    for wp, loc, _ in wps_near:
        yaw_rad = math.radians(wp.transform.rotation.yaw)
        nx = -math.sin(yaw_rad)
        ny =  math.cos(yaw_rad)
        half_width = wp.lane_width * 0.5
        half_length = step * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        # LEFT marking
        lm_left = wp.left_lane_marking
        if lm_left is not None and lm_left.type != carla.LaneMarkingType.NONE:
            bx = loc.x - nx * half_width
            by = loc.y - ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 1.0
                lm_type = float(int(lm_left.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,      # very thin strip
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

        # RIGHT marking
        lm_right = wp.right_lane_marking
        if lm_right is not None and lm_right.type != carla.LaneMarkingType.NONE:
            bx = loc.x + nx * half_width
            by = loc.y + ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 2.0
                lm_type = float(int(lm_right.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )
        # 3) Landmarks (optional)
    if all_landmarks is not None:
        for lm in all_landmarks:
            loc = lm.transform.location
            dx = loc.x - ego_loc.x
            dy = loc.y - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist > radius:
                continue

            yaw_rad = math.radians(lm.transform.rotation.yaw)
            half_length = 1.0
            half_width = 1.0

            kind = classify_landmark(lm)

            # geom_type = 3: "landmark-like object"
            geom_type_code = 3.0

            if kind == "stop_sign":
                subtype = 1.0
            elif kind == "yield_sign":
                subtype = 2.0
            elif kind == "traffic_light":
                subtype = 3.0
            else:
                subtype = 0.0  # other / ignore later if you want

            lm_id = float(lm.id)

            candidates.append(
                (
                    dist,
                    loc.x, loc.y,
                    yaw_rad,
                    half_length, half_width,
                    0.0, lm_id,        # lane_type=0 for landmarks
                    geom_type_code, subtype,
                )
            )

    if all_crosswalks is not None:
        for lm in all_crosswalks:
            loc = lm
            dx = loc.x - ego_loc.x
            dy = loc.y - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist > radius:
                continue

            #yaw_rad = math.radians(lm.transform.rotation.yaw)
            yaw_rad = 0 # set to rad as lm is just location
            # Rough size guesses; you can tune these per lm.type if you like
            half_length = 1.0
            half_width = 1.0

            geom_type_code = 4.0  # "crosswalks"
            subtype = 0.0
#             # landmark.type is often a numeric string like "1000001"
#             try:
#                 subtype = float(int(lm.type))
#             except (ValueError, TypeError):
#                 subtype = 0.0
            # Store landmark id in the "road_id" slot
            lm_id = 0.0

            candidates.append(
                (
                    dist,
                    loc.x, loc.y,
                    yaw_rad,
                    half_length, half_width,
                    0.0, lm_id,        # lane_type=0 for landmarks
                    geom_type_code, subtype,
                )
            )

    D = 9
    points = np.zeros((n_points, D), dtype=np.float32)
    if not candidates:
        return points

    # Sort by distance and keep up to n_points
    candidates.sort(key=lambda t: t[0])
    selected = candidates[:n_points]

    for i, (_, x, y, yaw, hlen, hwid,
            lane_t, rid, geom_t, lm_type) in enumerate(selected):
        points[i, :] = [x, y, yaw, hlen, hwid, lane_t, rid, geom_t, lm_type]

    return points


In [29]:
def get_local_points_from_precomputed_downsample(
    driving_wps,
    ego_loc,
    radius=60.0,
    n_points=200,
    step=2.0,
    all_landmarks=None,
):
    """
    Using precomputed driving waypoints:
      - Find waypoints whose *center* is within radius of ego_loc.
      - Build lane center points for them.
      - Build lane marking points (left/right) from them.
      - Keep only those boundary points whose own position is within radius.
      - Sort all candidates by distance.
      - If there are more than n_points, UNIFORMLY DOWNSAMPLE across distance.

    Returns:
        points: np.ndarray of shape (n_points, 9), rows:

            [x_world, y_world, yaw_rad,
             half_length, half_width,
             lane_type_code, road_id,
             geom_type_code, lane_mark_type_code]
    """
    candidates = []

    # First: waypoints whose center is within radius
    wps_near = []
    for wp in driving_wps:
        loc = wp.transform.location
        dx = loc.x - ego_loc.x
        dy = loc.y - ego_loc.y
        dist = math.hypot(dx, dy)
        if dist <= radius:
            wps_near.append((wp, loc, dist))

    # 1) Lane center points
    for wp, loc, dist in wps_near:
        yaw_rad = math.radians(wp.transform.rotation.yaw)
        half_length = step * 0.5
        half_width = wp.lane_width * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        geom_type_code = 0.0
        lm_type = 0.0

        candidates.append(
            (
                dist,
                loc.x, loc.y,
                yaw_rad,
                half_length, half_width,
                lane_type_code, road_id,
                geom_type_code, lm_type,
            )
        )

    # 2) Lane marking points (left/right)
    for wp, loc, _ in wps_near:
        yaw_rad = math.radians(wp.transform.rotation.yaw)
        nx = -math.sin(yaw_rad)
        ny =  math.cos(yaw_rad)
        half_width = wp.lane_width * 0.5
        half_length = step * 0.5
        lane_type_code = float(wp.lane_type)
        road_id = float(wp.road_id)

        # LEFT marking
        lm_left = wp.left_lane_marking
        if lm_left is not None and lm_left.type != carla.LaneMarkingType.NONE:
            bx = loc.x - nx * half_width
            by = loc.y - ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 1.0
                lm_type = float(int(lm_left.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

        # RIGHT marking
        lm_right = wp.right_lane_marking
        if lm_right is not None and lm_right.type != carla.LaneMarkingType.NONE:
            bx = loc.x + nx * half_width
            by = loc.y + ny * half_width
            dx = bx - ego_loc.x
            dy = by - ego_loc.y
            dist = math.hypot(dx, dy)
            if dist <= radius:
                geom_type_code = 2.0
                lm_type = float(int(lm_right.type))
                candidates.append(
                    (
                        dist,
                        bx, by,
                        yaw_rad,
                        half_length, 0.1,
                        lane_type_code, road_id,
                        geom_type_code, lm_type,
                    )
                )

    D = 9
    points = np.zeros((n_points, D), dtype=np.float32)
    if not candidates:
        return points

    # Sort by distance
    candidates.sort(key=lambda t: t[0])
    total = len(candidates)

    # Uniform downsampling across distance axis
    if total <= n_points:
        selected = candidates
    else:
        # indices spaced evenly from closest to farthest
        idxs = np.linspace(0, total - 1, n_points, dtype=int)
        selected = [candidates[i] for i in idxs]

    for i, (_, x, y, yaw, hlen, hwid,
            lane_t, rid, geom_t, lm_type) in enumerate(selected):
        points[i, :] = [x, y, yaw, hlen, hwid, lane_t, rid, geom_t, lm_type]

    # If total < n_points, remaining rows stay zero-padded
    return points


In [30]:
def draw_local_map_points_precomputed(
    world,
    driving_wps,
    all_landmarks,
    all_crosswalks,
    geo_location,
    radius=60.0,
    n_points=200,
    step=2.0,
    life_time=20.0,
    max_draw=None,
):
    """
    Same as before, but also draws landmarks (geom_type 3) in a distinct color.
    """
    if max_draw is None:
        max_draw = n_points

    carla_map = world.get_map()
    debug = world.debug

    ego_wp = carla_map.get_waypoint(
        geo_location,
        project_to_road=True,
        lane_type=carla.LaneType.Driving,
    )
    ego_loc = ego_wp.transform.location

    points = get_local_points_from_precomputed_knearest(
        driving_wps,
        ego_loc,
        radius=radius,
        n_points=n_points,
        step=step,
        all_landmarks=all_landmarks,
        all_crosswalks=all_crosswalks,
    )

    num_drawn = 0
    for row in points:
        x, y, yaw, hlen, hwid, lane_t, rid, geom_t, subtype = row
        if x == 0 and y == 0 and hlen == 0 and hwid == 0:
            continue

        loc = carla.Location(x=float(x), y=float(y), z=ego_loc.z + 0.5)

        if int(geom_t) == 0:
            color = carla.Color(0, 255, 0)       # lane center
        elif int(geom_t) == 1:
            color = carla.Color(0, 0, 255)       # left marking
        elif int(geom_t) == 2:
            color = carla.Color(255, 255, 0)     # right marking
        elif int(geom_t) == 3:
            color = carla.Color(0, 255, 255)     # landmarks (stop sign, etc.)
        else:
            color = carla.Color(255, 0, 0)       # crosswalks 

        debug.draw_point(
            loc,
            size=0.08,
            color=color,
            life_time=life_time,
        )

        num_drawn += 1
        if num_drawn >= max_draw:
            break

    debug.draw_point(
        ego_loc + carla.Location(z=0.7),
        size=0.15,
        color=carla.Color(0, 255, 255),
        life_time=life_time,
    )

    print(f"Drew {num_drawn} map points (centers + lane drawings + landmarks) and ego.")
    return points, ego_loc


In [55]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
carla_map = world.get_map()

STEP = 5.0
driving_wps = precompute_driving_waypoints(carla_map, step=STEP)
all_landmarks = precompute_landmarks(carla_map)
all_crosswalks = precompute_crosswalks(carla_map)

# pick some geo location (e.g., from ego vehicle)
all_wps_for_test = carla_map.generate_waypoints(distance=10.0)
ego_wp = random.choice(all_wps_for_test)

#geo_location = ego_wp.transform.location
ego_location = carla.Location(x=-105.1, y=19.5)

RADIUS = 50.0
N_POINTS = 200

points, ego_loc = draw_local_map_points_precomputed(
    world,
    driving_wps,
    all_landmarks,
    all_crosswalks,
    geo_location=ego_location,
    radius=RADIUS,
    n_points=N_POINTS,
    step=STEP,
    life_time=20.0,
    max_draw=200,
)

print("Points shape:", points.shape)
print("First 5 points:\n", points[:5])


Precomputed 2969 driving waypoints (step=5.0)
Precomputed 140 landmarks
Precomputed 365 crosswalks
Drew 200 map points (centers + lane drawings + landmarks) and ego.
Points shape: (200, 9)
First 5 points:
 [[-1.0518076e+02  2.1033134e+01  2.3505545e+00  2.5000000e+00
   1.7500000e+00  2.0000000e+00  4.4000000e+01  0.0000000e+00
   0.0000000e+00]
 [-1.0518077e+02  2.1033134e+01  2.3505545e+00  2.5000000e+00
   1.7500000e+00  2.0000000e+00  9.8000000e+02  0.0000000e+00
   0.0000000e+00]
 [-1.0267375e+02  2.0986628e+01 -3.9326305e+00  2.5000000e+00
   1.0000000e-01  2.0000000e+00  9.9000000e+02  2.0000000e+00
   2.0000000e+00]
 [-1.0642516e+02  1.9802696e+01  2.3505545e+00  2.5000000e+00
   1.0000000e-01  2.0000000e+00  4.4000000e+01  2.0000000e+00
   2.0000000e+00]
 [-1.0393637e+02  2.2263573e+01  2.3505545e+00  2.5000000e+00
   1.0000000e-01  2.0000000e+00  9.8000000e+02  1.0000000e+00
   2.0000000e+00]]


In [50]:
visualize_locations(world, all_crosswalks,50)


## Convert Carla points to GPUDRIVE

In [59]:
def wrap_to_pi(angle):
    """Wrap angle (radians) to [-pi, pi]."""
    return (angle + math.pi) % (2.0 * math.pi) - math.pi

def carla_points_to_gpudrive_roadgraph(
    points: np.ndarray,
    ego_location,
    ego_yaw_deg: float,
) -> np.ndarray:
    """
    Convert CARLA map points (world-frame) into GPUDrive-style local roadgraph features.

    Args:
        points: (N, 9) numpy array:
            [x_w, y_w, yaw_w,
             half_length, half_width,
             lane_type, road_id,
             geom_type, subtype]
        ego_location: carla.Location (or any object with .x and .y attributes)
        ego_yaw_deg: ego yaw in degrees (CARLA yaw, usually from waypoint rotation)

    Returns:
        features: (N, 13) numpy array:
            [pos_x_rel, pos_y_rel,
             seg_len, scale_x, scale_y,
             yaw_rel,
             one_hot[0..6]]  # None, RoadLine, RoadEdge, RoadLane, CrossWalk, Speedbump, Stopsign
    """
    assert points.shape[1] == 9, f"Expected (N, 9) points, got {points.shape}"

    ego_x = float(ego_location.x)
    ego_y = float(ego_location.y)
    ego_yaw_rad = math.radians(ego_yaw_deg)

    cos_e = math.cos(ego_yaw_rad)
    sin_e = math.sin(ego_yaw_rad)

    N = points.shape[0]
    out = np.zeros((N, 13), dtype=np.float32)

    for i in range(N):
        x_w, y_w, yaw_w, hlen, hwid, lane_t, rid, geom_t, subtype = points[i]

        # Treat fully-zero geometry as padding; leave output row as zeros.
        if hlen == 0.0 and hwid == 0.0 and x_w == 0.0 and y_w == 0.0:
            continue

        # 1) World -> ego local coordinates for position
        dx = x_w - ego_x
        dy = y_w - ego_y

        # Rotate by -ego_yaw
        x_rel = cos_e * dx + sin_e * dy
        y_rel = -sin_e * dx + cos_e * dy

        # 2) Relative orientation
        yaw_rel = wrap_to_pi(yaw_w - ego_yaw_rad)

        # 3) Decide semantic class from geom_type
        geom = int(round(geom_t))

        # Default: None
        class_id = 0
        seg_len = 0.0
        scale_x = 0.0
        scale_y = 0.0

        if geom == 0:
            # Lane center -> RoadLane
            class_id = 3  # RoadLane
            seg_len = float(hlen)          # half-length, like GPUDrive
            scale_x = 0.001                # thin line convention
            scale_y = 0.001
        elif geom in (1, 2):
            # Left/right lane marking -> RoadLine
            class_id = 1  # RoadLine
            seg_len = float(hlen)
            scale_x = 0.001
            scale_y = 0.001
        elif geom == 3:
            # Landmark → decide based on subtype
            st = int(round(subtype))
            if st == 1:
                # stop_sign
                class_id = 6  # Stopsign
                seg_len = 0.002
                scale_x = 0.002
                scale_y = 0.001
            else:
                # yield, traffic lights, others → currently ignored in GPUDrive classes
                class_id = 0  # None
                seg_len = 0.0
                scale_x = 0.0
                scale_y = 0.0

        elif geom == 4:
            # Crosswalk: for now, "set aside" -> None class
            class_id = 0
            seg_len = 0.0
            scale_x = 0.0
            scale_y = 0.0

        # 4) One-hot encoding
        one_hot = np.zeros(7, dtype=np.float32)
        if 0 <= class_id < 7:
            one_hot[class_id] = 1.0

        # 5) Pack into output
        out[i, 0:6] = [
            x_rel,
            y_rel,
            seg_len,
            scale_x,
            scale_y,
            yaw_rel,
        ]
        out[i, 6:] = one_hot

    return out

In [60]:
ego_wp = carla_map.get_waypoint(
    ego_loc,
    project_to_road=True,
    lane_type=carla.LaneType.Driving,
)
ego_yaw_deg = ego_wp.transform.rotation.yaw

rg_features = carla_points_to_gpudrive_roadgraph(
    points,               # (200, 9) from your sampler
    ego_location=ego_loc,
    ego_yaw_deg=ego_yaw_deg,
)


print("Roadgraph features shape:", rg_features.shape)  # (200, 13)
print("First row:", rg_features[0])


Roadgraph features shape: (200, 13)
First row: [ 1.1700084e+00 -3.0202323e-06  2.5000000e+00  1.0000000e-03
  1.0000000e-03  1.8611784e-08  0.0000000e+00  0.0000000e+00
  0.0000000e+00  1.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00]


In [62]:
import pickle
file_path = "sample_features.pkl"
with open(file_path, 'wb') as file:
    pickle.dump(rg_features, file)
print(f"Variable saved to {file_path}")

Variable saved to sample_features.pkl


In [63]:
for item in rg_features:
    if item[12] != 0.0:
        print(item)

### Debug: visualize all landmarks

In [39]:
import carla

# Connect (adjust host/port if needed)
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)

world = client.get_world()
carla_map = world.get_map()
debug = world.debug

landmarks = carla_map.get_all_landmarks()
print(f"Found {len(landmarks)} landmarks")

LIFE_TIME = 200.0

for lm in landmarks:
    loc = lm.transform.location

    # Big fat point
    debug.draw_point(
        loc + carla.Location(z=1.5),
        size=0.3,
        color=carla.Color(255, 0, 255),  # bright magenta
        life_time=LIFE_TIME,
    )

    # Label with id + type
    debug.draw_string(
        loc + carla.Location(z=2.0),
        text=f"{lm.id} | {lm.type}",
        draw_shadow=True,
        color=carla.Color(255, 255, 255),
        life_time=LIFE_TIME,
    )

print("Landmarks drawn for 200 seconds. Fly around with the spectator and look for magenta dots + labels.")


Found 68 landmarks
Landmarks drawn for 200 seconds. Fly around with the spectator and look for magenta dots + labels.


In [38]:
import carla
from collections import Counter

client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
carla_map = world.get_map()

landmarks = carla_map.get_all_landmarks()
print("Total landmarks:", len(landmarks))

counter = Counter((lm.type, lm.name) for lm in landmarks)
for (lm_type, lm_name), count in counter.most_common():
    print(f"{count:3d} x type={lm_type!r}, name={lm_name!r}")


Total landmarks: 68
 56 x type='1000001', name='Signal_3Light_Post01'
  6 x type='206', name='Sign_Stop'
  3 x type='206', name='Stencil_STOP'
  2 x type='1000001', name=''
  1 x type='205', name='Sign_Yield'


In [45]:

def visualize_locations(world: carla.World,
                        locations,
                        life_time: float = 5.0):
    """
    Draws all given locations as points in the world for `life_time` seconds.
    """
    debug = world.debug
    color = carla.Color(255, 0, 0)  # red dots
    size = 0.2                      # radius in meters

    for loc in locations:
        debug.draw_point(
            loc,
            size=size,
            color=color,
            life_time=life_time,
            persistent_lines=False
        )
